In [47]:
!pip install pycoingecko
!pip install plotly
!pip install mplfinance

In [48]:
import pandas as pd
import plotly.graph_objects as go
from plotly.offline import plot
import matplotlib.pyplot as plt
import datetime
from pycoingecko import CoinGeckoAPI

In [49]:
cg= CoinGeckoAPI()
bitcoin_data = cg.get_coin_market_chart_by_id(id='bitcoin', vs_currency='usd', days=30)
print(type(bitcoin_data))
bitcoin_data.keys()

<class 'dict'>


dict_keys(['prices', 'market_caps', 'total_volumes'])

In [50]:
#bitcoin_data is a dictionary so we take all the values un 'Prices' label
bitcoin_price_data = bitcoin_data['prices']
bitcoin_price_data[0:5]

[[1657018823941, 19754.983833111804],
 [1657022455809, 19562.043240132836],
 [1657026138517, 19420.07949592582],
 [1657029724509, 19445.298225188257],
 [1657033353371, 19472.913399097648]]

In [51]:
#bitcoin_price_data has the price with correlated time of the price so we make dataframe with 'Timestamp' and 'Price'
data=pd.DataFrame(bitcoin_price_data, columns=['TimeStamp', 'Price'])
data

,TimeStamp,Price
0,1657018823941,19754.983833
1,1657022455809,19562.043240
2,1657026138517,19420.079496
3,1657029724509,19445.298225
4,1657033353371,19472.913399
...,...,...
716,1659596591668,22941.334257
717,1659600077061,22966.697985
718,1659603913638,22840.295983
719,1659607260011,22872.478095


In [52]:
#Convert the values from 'TimeStamp' to a readable data format with pd.to_datetime
data['Date'] = pd.to_datetime(data['TimeStamp'], unit ='ms')
data

,TimeStamp,Price,Date
0,1657018823941,19754.983833,2022-07-05 11:00:23.941
1,1657022455809,19562.043240,2022-07-05 12:00:55.809
2,1657026138517,19420.079496,2022-07-05 13:02:18.517
3,1657029724509,19445.298225,2022-07-05 14:02:04.509
4,1657033353371,19472.913399,2022-07-05 15:02:33.371
...,...,...,...
716,1659596591668,22941.334257,2022-08-04 07:03:11.668
717,1659600077061,22966.697985,2022-08-04 08:01:17.061
718,1659603913638,22840.295983,2022-08-04 09:05:13.638
719,1659607260011,22872.478095,2022-08-04 10:01:00.011


In [53]:
#Prepare the dataframe for candlestick plotting by using groupby using 'Date'
candlestick_data = data.groupby(data.Date.dt.date).agg({"Price": ['min', 'max', 'first', 'last']})
candlestick_data

Price                                          
                     min           max         first          last
Date                                                              
2022-07-05  19420.079496  20493.562679  19754.983833  20375.983039
2022-07-06  19885.772103  20517.772667  20188.126382  20517.772667
2022-07-07  20347.039132  21790.773519  20566.687224  21607.390054
2022-07-08  21293.230607  22109.673639  21660.596837  21766.456667
2022-07-09  21511.656552  21851.623147  21767.370196  21634.778796
2022-07-10  20828.123959  21600.467473  21600.467473  20872.106768
2022-07-11  20062.918477  20867.403980  20867.403980  20062.918477
2022-07-12  19415.440969  20027.259080  19961.625901  19415.440969
2022-07-13  19113.688679  19912.101239  19350.737350  19844.246945
2022-07-14  19748.348514  20697.657307  20204.492326  20499.224347
2022-07-15  20435.357941  21135.784033  20594.754617  20889.314292
2022-07-16  20522.749340  21399.149591  20794.811981  21227.184606
2022-07-17  20935.721864  21509.626029  21193.083343  20995.891185
2022-07-18  20802.939872  22343.508744  20802.939872  21784.494533
2022-07-19  21766.862634  23654.858145  22395.365383  23461.983919
2022-07-20  22993.315870  24160.260513  23366.907698  23364.053665
2022-07-21  22524.740214  23274.063474  23216.590217  23016.031815
2022-07-22  22648.696277  23665.847268  23155.207555  22737.283714
2022-07-23  22067.867506  22913.847008  22696.902754  22511.787846
2022-07-24  22338.578694  22860.154563  22506.199574  22860.154563
2022-07-25  21661.450773  22606.217112  22606.217112  21661.450773
2022-07-26  20783.387306  21401.478074  21401.478074  21106.480350
2022-07-27  21079.370755  22855.894271  21235.612316  22855.894271
2022-07-28  22777.251048  24096.392735  22908.884260  23946.941971
2022-07-29  23562.502044  24111.642706  23848.954756  23944.490130
2022-07-30  23729.448125  24581.147996  23857.805954  23729.448125
2022-07-31  23288.617189  23858.138350  23653.459549  23288.617189
2022-08-01  22916.219055  23467.475085  23344.855867  23188.233857
2022-08-02  22754.248682  23412.928605  23304.157193  23115.382086
2022-08-03  22778.669545  23561.825256  23052.946433  23026.022874
2022-08-04  22840.295983  23165.072999  22860.420984  22947.371259

In [55]:
fig = go.Figure(data=[go.Candlestick(x=candlestick_data.index,
                open=candlestick_data['Price']['first'], 
                high=candlestick_data['Price']['max'],
                low=candlestick_data['Price']['min'], 
                close=candlestick_data['Price']['last'])
                ])

fig.update_layout(xaxis_rangeslider_visible=False, xaxis_title='Date', yaxis_title='Price ($USD)' , title= 'Bitcoin Candlestick Chart past 30 days')

fig.show()

In [56]:
plot(fig, filename='bitcoin_candlestick_chart.html')

'bitcoin_candlestick_chart.html'